<a href="https://colab.research.google.com/github/muhammadalfateh6/-ALGORYC_Rep/blob/main/Yolo8_5000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import numpy as np 
import pandas as pd 

from pathlib import Path
from xml.dom.minidom import parse
from shutil import copyfile
import os

In [5]:
classes = ['helmet','head','person']

In [6]:
def convert_annot(size , box):
    x1 = int(box[0])
    y1 = int(box[1])
    x2 = int(box[2])
    y2 = int(box[3])

    dw = np.float32(1. / int(size[0]))
    dh = np.float32(1. / int(size[1]))

    w = x2 - x1
    h = y2 - y1
    x = x1 + (w / 2)
    y = y1 + (h / 2)

    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return [x, y, w, h]

In [7]:
def save_txt_file(img_jpg_file_name, size, img_box):
    save_file_name = '/content/drive/MyDrive/HHD_Project/Dataset/labels/' +  img_jpg_file_name + '.txt'
    
    #file_path = open(save_file_name, "a+")
    with open(save_file_name ,'a+') as file_path:
        for box in img_box:

            cls_num = classes.index(box[0])

            new_box = convert_annot(size, box[1:])

            file_path.write(f"{cls_num} {new_box[0]} {new_box[1]} {new_box[2]} {new_box[3]}\n")

        file_path.flush()
        file_path.close()

In [8]:
def get_xml_data(file_path, img_xml_file):
    img_path = file_path + '/' + img_xml_file + '.xml'
    #print(img_path)

    dom = parse(img_path)
    root = dom.documentElement
    img_name = root.getElementsByTagName("filename")[0].childNodes[0].data
    img_size = root.getElementsByTagName("size")[0]
    objects = root.getElementsByTagName("object")
    img_w = img_size.getElementsByTagName("width")[0].childNodes[0].data
    img_h = img_size.getElementsByTagName("height")[0].childNodes[0].data
    img_c = img_size.getElementsByTagName("depth")[0].childNodes[0].data
   
    img_box = []
    for box in objects:
        cls_name = box.getElementsByTagName("name")[0].childNodes[0].data
        x1 = int(box.getElementsByTagName("xmin")[0].childNodes[0].data)
        y1 = int(box.getElementsByTagName("ymin")[0].childNodes[0].data)
        x2 = int(box.getElementsByTagName("xmax")[0].childNodes[0].data)
        y2 = int(box.getElementsByTagName("ymax")[0].childNodes[0].data)
        
        img_jpg_file_name = img_xml_file + '.jpg'
        img_box.append([cls_name, x1, y1, x2, y2])
  

    # test_dataset_box_feature(img_jpg_file_name, img_box)
    save_txt_file(img_xml_file, [img_w, img_h], img_box)

In [10]:
files = os.listdir('/content/drive/MyDrive/HHD_Project/Dataset_kaggle/labels_5000')
for file in files:
    file_xml = file.split(".")
    get_xml_data('/content/drive/MyDrive/HHD_Project/Dataset_kaggle/labels_5000', file_xml[0])

In [11]:
from sklearn.model_selection import train_test_split
image_list = os.listdir('/content/drive/MyDrive/HHD_Project/Dataset_kaggle/images_5000')
train_list, test_list = train_test_split(image_list, test_size=0.2, random_state=42)
val_list, test_list = train_test_split(test_list, test_size=0.5, random_state=42)
print('total =',len(image_list))
print('train :',len(train_list))
print('val   :',len(val_list))
print('test  :',len(test_list))

total = 5000
train : 4000
val   : 500
test  : 500


In [12]:
def copy_data(file_list, img_labels_root, imgs_source, mode):

    root_file = Path('/content/drive/MyDrive/HHD_Project/Dataset/images/'+  mode)
    if not root_file.exists():
        print(f"Path {root_file} does not exit")
        os.makedirs(root_file)

    root_file = Path('/content/drive/MyDrive/HHD_Project/Dataset/labels/' + mode)
    if not root_file.exists():
        print(f"Path {root_file} does not exit")
        os.makedirs(root_file)

    for file in file_list:               
        img_name = file.replace('.png', '')        
        img_src_file = imgs_source + '/' + img_name + '.png'        
        label_src_file = img_labels_root + '/' + img_name + '.txt'

        #print(img_sor_file)
        #print(label_sor_file)
        # im = Image.open(rf"{img_sor_file}")
        # im.show()

        # Copy image
        DICT_DIR = '/content/drive/MyDrive/HHD_Project/Dataset/images/'  + mode
        img_dict_file = DICT_DIR + '/' + img_name + '.png'

        copyfile(img_src_file, img_dict_file)

        # Copy label
        DICT_DIR = '/content/drive/MyDrive/HHD_Project/Dataset/labels/' + mode
        img_dict_file = DICT_DIR + '/' + img_name + '.txt'
        copyfile(label_src_file, img_dict_file)

In [13]:
copy_data(train_list, '/content/drive/MyDrive/HHD_Project/Dataset/labels', '/content/drive/MyDrive/HHD_Project/Dataset_kaggle/images_5000', "train")
copy_data(val_list,   '/content/drive/MyDrive/HHD_Project/Dataset/labels', '/content/drive/MyDrive/HHD_Project/Dataset_kaggle/images_5000', "val")
copy_data(test_list,  '/content/drive/MyDrive/HHD_Project/Dataset/labels', '/content/drive/MyDrive/HHD_Project/Dataset_kaggle/images_5000', "test")

Path /content/drive/MyDrive/HHD_Project/Dataset/images/train does not exit
Path /content/drive/MyDrive/HHD_Project/Dataset/labels/train does not exit
Path /content/drive/MyDrive/HHD_Project/Dataset/images/val does not exit
Path /content/drive/MyDrive/HHD_Project/Dataset/labels/val does not exit
Path /content/drive/MyDrive/HHD_Project/Dataset/images/test does not exit
Path /content/drive/MyDrive/HHD_Project/Dataset/labels/test does not exit


In [16]:
%cd /content/drive/MyDrive/HHD_Project
!git clone https://github.com/ultralytics/ultralytics

/content/drive/MyDrive/HHD_Project
Cloning into 'ultralytics'...
remote: Enumerating objects: 10324, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 10324 (delta 42), reused 46 (delta 16), pack-reused 10235
Receiving objects: 100% (10324/10324), 6.42 MiB | 9.43 MiB/s, done.
Resolving deltas: 100% (7038/7038), done.
Updating files: 100% (367/367), done.


In [17]:
!pip install ultralytics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.5/593.5 kB 9.3 MB/s eta 0:00:00


In [18]:
%cd /content/drive/MyDrive/HHD_Project

import yaml

config = {
   "path": "/content/drive/MyDrive/HHD_Project/Dataset/images",
   "train": "train",
   "val": "val",
   "test": "test",
   "nc": 3,
   "names": ['helmet','head','person']
}
with open("data.yaml", "w") as file:
   yaml.dump(config, file, default_flow_style=False)

/content/drive/MyDrive/HHD_Project


In [ ]:
!yolo task=detect mode=train data=/content/drive/MyDrive/HHD_Project/data.yaml model=yolov8s.pt epochs=2 lr0=0.01

100% 21.5M/21.5M [00:00<00:00, 87.9MB/s]
Ultralytics YOLOv8.0.113 🚀 Python-3.10.11 torch-2.0.1+cu118 CPU
yolo/engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/HHD_Project/data.yaml, epochs=2, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, o